# 实验一
自定义深度卷积申请网络，并在Kaggle猫/狗数据集上进行训练和测试

## 1.加载keras模块

In [1]:
#加载ImageDataGenerator模块
from keras.preprocessing.image import ImageDataGenerator
#加载定义CNN的layers、activate function等模块
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D,Activation
#以下为将预测score转换为类别信息的to_categorical、图像预处理等函数
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
import numpy as np
import pandas as pd

Using TensorFlow backend.


### 定义CNN网络结构
首先尝试：自定义深度CNN结构

然后尝试：定义Alex/VGG网络结构


In [33]:
#首先定义图像的长和宽，目的是告诉网络输入图像矩阵的大小，便于搭建网络结构
img_width, img_height=150,150
#以下为确认图像的通道分量
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
#在此定义网络结构    
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),input_shape=input_shape,activation='relu'))
# model.add(Activation('relu'))
model.add(MaxPool2D((2,2),strides=2))
# model.add(Dropout(0.2))

model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
# model.add(Activation('relu'))
model.add(MaxPool2D((2,2),strides=2))
# model.add(Dropout(0.2))

model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
# model.add(Activation('relu'))
model.add(MaxPool2D((2,2),strides=2))
# model.add(Dropout(0.1))
model.add(Flatten())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))
#在此调用compile函数定义loss function，optimizer和metrics
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

### 查看model架构



In [34]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 18496)            

### 定义ImageDataGenerator


In [35]:
#指定train和validation数据路径
train_data_dir=r'D:\kejian\11.06\dogs-vs-cats\train'
validation_data_dir=r'D:\kejian\11.06\dogs-vs-cats\validation'
#指定train和validation数据规模
nb_train_samples=10500
nb_validation_samples=4500

epochs = 1
batch_size = 20

#声明ImageDataGenerator类型的train_datagen用于对training图像进行数据像素归一化和扩增
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

#声明ImageDataGenerator类型的test_datagen用于对validation图像进行像素归一化
test_datagen=ImageDataGenerator(rescale=1./255)
#调用flow_from_directory生成train_generator
train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=batch_size,
                                                  class_mode='binary')
#调用flow_from_directory生成validation_generator
validation_generator=test_datagen.flow_from_directory(validation_data_dir,
                                                      target_size=(img_width, img_height),
                                                      batch_size=batch_size,
                                                      class_mode='binary')



Found 10835 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


### 训练模型



In [37]:
#调用并设置fit_generator函数训练模型
sp=model.fit_generator(train_generator,
                       steps_per_epoch=nb_train_samples//batch_size,
                       epochs=epochs,
                       validation_data=validation_generator,
                       validation_steps=nb_validation_samples//batch_size)

Epoch 1/1
525/525 [==============================] - 104s 198ms/step - loss: 0.5826 - accuracy: 0.6962 - val_loss: 0.4605 - val_accuracy: 0.7140


### 使用训练后模型预测图像





In [38]:
import cv2
#调用opencv的imread函数和resize函数读取测试样本并进行尺寸缩放，此处路径可更换
img = cv2.resize(cv2.imread(r'D:\kejian\11.06\dogs-vs-cats\test\7.jpg'), (img_width, img_height)).astype(np.float32)

#对图像像素矩阵进行tensor类型转换
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
#用训练好的模型对测试样本进行预测
score=model.predict(x)
print(score)


[[0.]]
